In [1]:
import pandas as pd
import regex as re

In [2]:
file_rzepa = 'Rzeczpospolita 2015-2022.xlsx'
file_gpc = 'excels/Gazeta Polska Codziennie 2015-2022.xlsx'
file_newsweek = 'excels/Newsweek.xlsx'
file_wprost = "excels/wprost-2015.xlsx"
cols_rzepa = ['title', 'text', 'source', 'author','link','department','imgage','image_text']
cols_gpc = ['title', 'text', 'source', 'author','link','department','imgage']
cols_newsweek = ['title', 'text']

In [3]:
data = pd.read_excel(file_rzepa)

In [4]:
data['source'].value_counts()

source
Rzeczpospolita    216163
Name: count, dtype: int64

In [5]:
# Newsweek
#data.rename(columns={'title_main': 'title', 'author 1': 'author', "Wydanie": "department"}, inplace=True)

In [6]:
def data_pipeline(data, cols):
    data = data[data['text'].notna()]
    data.loc[~data['title'].apply(lambda x: isinstance(x, str)), 'title'] = "None"  # newsweek

    for col in cols:
        data[col] = data[col].fillna('None')

    CLEAN_XLSX_1 = re.compile('[(∑)]')
    REPLACE_EXCESS_SPACES_RE = re.compile('\s\s+')
    data['text'] = data['text'].apply(lambda x: re.split('(?i)©℗\s*Więcej\s*na', x)[0])
    for col in cols:
        data[col] = data[col].apply(lambda x: REPLACE_EXCESS_SPACES_RE.sub(' ', x))
        data[col] = data[col].apply(lambda x: CLEAN_XLSX_1.sub('', x))
        data[col] = data[col].apply(lambda x: x.replace("\xad", ""))
        data[col] = data[col].apply(lambda x: x.replace("\xa0", " "))
        data[col] = data[col].apply(lambda x: x.replace("_x0007_", " "))


    data['department'] = data['department'].str.strip()
    data['author'] = data['author'].str.strip()
    return data

In [7]:
def author_extract(text):
    extracted = re.search('((?<=\\t—).*)|(.*(?=@rp\.pl))', text)
    return extracted[0] if extracted is not None else extracted


def restore_authors(df):
    df['last'] = df['text'].apply(lambda x: x.split(' ')[-1])
    df['extracted'] = df['last'].apply(author_extract)
    df.loc[df['author']=='None', 'author'] = df["extracted"]
    df.drop(columns = ['last', 'extracted'], inplace = True)
    return df

def dot_problem_solver(text):
    THREE_DOT_RE = re.compile('(?<=[^\s])\.\.\.(?=[^\s])')
    text = THREE_DOT_RE.sub('... ', text)
    DOT_RE = re.compile('(?<=[^\s\.])\.(?=[^\s\.])')
    text = DOT_RE.sub('. ', text)
    QUEST_RE = re.compile('(?<=[^\s])\?(?=[^\s])')
    text = QUEST_RE.sub('? ', text)
    return text

In [8]:
def splitter(text):
    re_expr = "([A-Z]*[a-z]+)([A-Z][a-z]*)"
    matches = re.findall(re_expr, text)
    if matches != None:
        for match in matches:
            text_found = "".join(match)
            to_replace = match[0] + " " + match[1]
            text = text.replace(text_found, to_replace)
    return text


In [9]:
data = data_pipeline(data, cols_newsweek)

/tmp/ipykernel_3248/2377280704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].fillna('None')
/tmp/ipykernel_3248/2377280704.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: re.split('(?i)©℗\s*Więcej\s*na', x)[0])
/tmp/ipykernel_3248/2377280704.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [10]:
#data_all = restore_authors(data_all) # Rzepa
data['text'] = data['text'].apply(dot_problem_solver)


In [15]:
!mkdir chunks

In [17]:
re.split("twoja stara", "xd xd haha nie ma jej tu")

['xd xd haha nie ma jej tu']

In [16]:
from math import ceil

def chunkit(df, chunk_size= 10000):
    num_chunks = ceil(len(df) / chunk_size)
    for i in range(num_chunks):
        yield df[i*chunk_size:(i + 1) * chunk_size]

for i, chunk in enumerate(chunkit(data)): chunk.to_csv(f"chunks/rzepa_{i}.csv", index=False)

In [18]:
!rm -r chunks

In [19]:
data.to_csv("rzepa_all.csv", index=False)